In [1]:
import tensorflow as tf

In [16]:
# Check for the number of images in a folder
import os
food_image_paths = []
for dir, sub_dir, files in os.walk("data_small_test\\train"):
    for file in files:
        food_image_paths.append(os.path.join(dir,file))
len(food_image_paths)

1600

In [3]:
train_dir = "data_small_test\\train"
test_dir = "data_small_test\\test"
train_dir, test_dir

('data_small_test\\train', 'data_small_test\\test')

In [4]:
# Load in data
train_data = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
            batch_size=32,
            image_size=(224,224)
            )

test_data = tf.keras.preprocessing.image_dataset_from_directory(test_dir,
            batch_size=32,
            image_size=(224,224)
            )

train_data, test_data

Found 1600 files belonging to 2 classes.
Found 400 files belonging to 2 classes.


(<BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>,
 <BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>)

In [5]:
train_data.class_names

['apple_pie', 'baby_back_ribs']

In [6]:
train_data = train_data.prefetch(tf.data.AUTOTUNE)
test_data = test_data.prefetch(tf.data.AUTOTUNE)

In [7]:
base_model = tf.keras.applications.EfficientNetB0(include_top=False)

# Make model untrainable
base_model.trainable = False

# Build a functional model
input_layer = tf.keras.Input(shape=(224,224,3))
x = base_model(input_layer)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
output_layer = tf.keras.layers.Dense(1, activation="sigmoid")(x)

# Construct model
model_1 = tf.keras.Model(input_layer, output_layer, name="EfficientNetB0-V1")

In [9]:
# tf.keras.utils.plot_model(model_1, show_shapes=True)

FileNotFoundError: [Errno 2] No such file or directory: 'model.png'

In [10]:
# Compile model
model_1.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                 optimizer= tf.keras.optimizers.Adam(),
                 metrics=["accuracy"])

In [11]:
# Create EarlyStopping callback and TensorBoard callback
early_stopping = tf.keras.callbacks.EarlyStopping(patience=5,
                                                  monitor="val_loss")

In [12]:
# Fit model
history_1 = model_1.fit(train_data,
                        epochs=3,
                        validation_data=test_data,
                        callbacks=[early_stopping]
                        ) # Adjust for imbalance data

Epoch 1/3
50/50 [==============================] - 93s 2s/step - loss: 0.3887 - accuracy: 0.8900 - val_loss: 0.2210 - val_accuracy: 0.9675
Epoch 2/3
50/50 [==============================] - 82s 2s/step - loss: 0.1772 - accuracy: 0.9694 - val_loss: 0.1425 - val_accuracy: 0.9775
Epoch 3/3
50/50 [==============================] - 83s 2s/step - loss: 0.1276 - accuracy: 0.9762 - val_loss: 0.1115 - val_accuracy: 0.9800


In [13]:
model_1.evaluate(test_data)

13/13 [==============================] - 17s 1s/step - loss: 0.1115 - accuracy: 0.9800


[0.1114860251545906, 0.9800000190734863]

## Save and convert model to tfjs model

In [10]:
# Convert tfjs model
import tensorflowjs as tfjs
import tensorflow as tf


model = tf.keras.models.load_model("model\\food_not_food_model_B0-V1.h5")

tfjs.converters.save_keras_model(model, "model\\tfjs_model")

weight normalization/count with shape () and dtype int64 was auto converted to the type int32


NotFoundError: Failed to create a NewWriteableFile: model\tfjs_model\group1-shard1of4.bin : The system cannot find the file specified.
; No such file or directory